# Used to expand a BERT-style checkpoint if it doesn't have enough "unused" token required to expand the vocab

### Note: Restart the Kernel after every model reassignment
###       Use Tensorflow v1 for this notebook

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf
import numpy as np

#### Parameters for training

In [1]:
old_vocab_size  =30522
additional_vocab=30000
bert_dimension  =768
input_model_path=""     ## Sample: '/model_path/model_dir/bert_model.ckpt'
out_dir=""              ## Sample: '/model_path/model_out_dir/'


## Expansion Code

In [ ]:
new_vocab_size=old_vocab_size+additional_vocab
checkpoint = {}
with tf.compat.v1.Session() as sess:
    saver = tf.train.import_meta_graph(input_model_path+'.meta')
    saver.restore(sess, input_model_path)
    print(len(tf.global_variables()))
    for x in tf.global_variables():
        print(str(x.name),x.eval().shape)
        if(x.name=='bert/embeddings/word_embeddings:0'):
            print("Changing bert/embeddings/word_embeddings:0")
            new_embed=np.random.rand(new_vocab_size,bert_dimension)
            new_embed[:old_vocab_size,:]=x.eval()
            sess.run(tf.assign(x,new_embed, validate_shape=False))
        if(x.name=='cls/predictions/output_bias:0'):
            print("Changing cls/predictions/output_bias:0 ")
            new_bias=np.random.rand(new_vocab_size)
            new_bias[:old_vocab_size]=x.eval()
            sess.run(tf.assign(x,new_bias, validate_shape=False))

        print(str(x.name),x.eval().shape)
    saver.save(sess,out_dir+'/expanded_model.ckpt')    
